In [2]:
import weaviate
from weaviate.config import AdditionalConfig
from weaviate.classes.config import Configure

import json
import pandas as pd
import numpy as np
import os
import dotenv

dotenv.load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')
AZURE_OPENAI_RESOURCE_NAME = os.getenv('AZURE_OPENAI_RESOURCE_NAME')


client = weaviate.connect_to_local(
    port=8083,
    grpc_port=50051,
    # headers = {
    #     "X-Azure-Api-Key": AZURE_OPENAI_API_KEY,
    # },
)

print(client.is_ready())


True


In [ ]:
data = pd.read_parquet('/home/isma/repos/book/data-pipelines-with-airflow-2nd-ed/chapter13_genai/recipe_book/notebooks/splitted_with_vectors.parquet')
data

In [ ]:
COLLECTION_NAME = "recipes"

existing_collections = [item.lower() for item in list(client.collections.list_all().keys())]

if COLLECTION_NAME.lower() in existing_collections:

    client.collections.delete(COLLECTION_NAME)


collection = client.collections.create(
    COLLECTION_NAME,
    vectorizer_config=[
        Configure.NamedVectors.text2vec_azure_openai(
            name="vector",
            source_properties=["chunk"],
            base_url= AZURE_OPENAI_ENDPOINT,
            resource_name=AZURE_OPENAI_RESOURCE_NAME,
            deployment_id="text-embedding-3-large",
        )    
    ],





)

collection.config.get().to_dict()


In [ ]:
COLLECTION_NAME = "recipes"
collection = client.collections.get(COLLECTION_NAME)


source_objects = json.loads( 
    data
    .drop(columns=["document_sha"])
    .to_json(orient="records")
) 


with collection.batch.dynamic() as batch:
    for src_obj in source_objects:
        properties_obj = {
            "filename": src_obj["filename"],
            "chunk": src_obj["chunk"],
        }

        # The model provider integration will automatically vectorize the object
        batch.add_object(
            properties=properties_obj,
            uuid=src_obj["chunk_sha"],
        )


In [ ]:

response = collection.query.near_text(
    query="Como hacer empanadas",  # The model provider integration will automatically vectorize the query
    limit=2
)

for obj in response.objects:
    print(obj.properties["chunk"])

In [ ]:
for item in collection.iterator():
    print(item.uuid, item.properties)

In [ ]:
collection.data.delete_by_id(
    '9c95627f-1e0c-5a5d-8bdd-eb4e9cbb7da0'
)

In [3]:
COLLECTION_NAME = "recipes"
collection = client.collections.get(COLLECTION_NAME)

In [4]:
for item in collection.iterator():
    print(item.uuid, item.properties)

0005946b-0fc2-5304-8e81-fc6c09bd1fc0 {'filename': 'empanadas-argentinas.txt', 'chunk_sha': UUID('0005946b-0fc2-5304-8e81-fc6c09bd1fc0'), 'chunk': 'de harina, para extender la masa suficiente de huevo, para barnizar chimichurri al gusto, para acompañar Preparación En un bowl, mezcla la harina, la sal, polvo para hornear, la mantequilla y el vinagre hasta integrar muy bien la mantequilla, después vierte el agua poco a poco y mezcla hasta formar', 'document_sha': UUID('ec598303-9510-5d6e-9c5e-ccb43fcf8c4e')}
035ecfda-a0a7-5c91-b708-f8db7dee9d21 {'filename': 'empanadas-argentinas.txt', 'chunk_sha': UUID('035ecfda-a0a7-5c91-b708-f8db7dee9d21'), 'chunk': 'Empanadas argentinas   Ingredientes Medidas 4 Porciones Seleccionar todos los ingredientes 2 tazas de harina de trigo, para la masa 1 cucharadita de sal, para la masa 1 cucharadita de polvo para hornear, para la masa 1/2 tazas de mantequilla, en cubos y a temperatura ambiente, para la masa 1/4 tazas', 'document_sha': UUID('ec598303-9510-5d6

In [ ]:
client.collections.delete(COLLECTION_NAME)